![image info](BCI.png)

# Classification of brain states (FOOT and HAND Movement) framework
--------------------------------------------------------------------------------------------------------------------------------

### Data
i. The framework uses the dataset of healthy control subjects performing imagery right and left hand movement.

ii. The total number of signals corresponds to RIGHT HAND MOVEMENT is 500000 while for RIGHT FOOT MOVEMENT it was 400000. 

iii. Each class has 118 channels.

iv. Initial sampling rate = 1KHz, i.e. 1000 samples are captured in one second.

v. The data is downsampled to 100 Hz,  i.e. 100 samples in one second.

## Import required libraries (Note import is repeated to understand the required libraries are imported when specific module is implemented)

In [ ]:
# Importing required libraries required to execution
import numpy as np # linear algebra
print(np.version.version)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for statistical data visualization
%matplotlib inline
import scipy.io
import warnings
warnings.filterwarnings('ignore') # ignoring warnings if any just to avoid confusion during the execution

### Load the data stored in .MAT file and assign respective classes

In [ ]:
# Load the data which is stored in MAT file
BCI_data = scipy.io.loadmat('bci.mat')
RH = BCI_data['rh'] # Signals belonging to right hand (RH) movement
RF = BCI_data['rf'] # Signals belonging to right foot (RF) movement

In [ ]:
# Find NaN values in matrix2
nan_mask = np.isnan(RF)  # Boolean mask where NaNs are present

# Get row and column indices of NaNs
nan_positions = np.argwhere(nan_mask)  # Returns list of (row, col) positions

# Find which columns contain NaNs
nan_columns = np.where(np.any(nan_mask, axis=0))[0]  # Columns with NaN values

# Print results
# Print results
print(f"NaN values found in {len(nan_positions)} locations.")
print(f"Columns with NaN values: {nan_columns.tolist()}")
print(f"First 10 NaN positions (row, col): {nan_positions[:10].tolist()}")  # Show first 10 for brevity

In [ ]:
# Function to impute NaN values with column mean (for median imputation just replace nanmedian)
def impute_with_mean(matrix):
    # Find NaN values
    nan_mask = np.isnan(matrix)
    
    # Calculate column means (ignoring NaNs)
    col_means = np.nanmedian(matrix, axis=0)
    
    # Replace NaNs with corresponding column mean
    matrix[nan_mask] = np.take(col_means, np.where(nan_mask)[1])
    
    return matrix

# Impute NaN values in both matrix1 and matrix2
RH = impute_with_mean(RH)
RF = impute_with_mean(RF)

### Verify if there are nan values

In [ ]:
# Find NaN values in matrix2
nan_mask = np.isnan(RF)  # Boolean mask where NaNs are present

# Get row and column indices of NaNs
nan_positions = np.argwhere(nan_mask)  # Returns list of (row, col) positions

# Find which columns contain NaNs
nan_columns = np.where(np.any(nan_mask, axis=0))[0]  # Columns with NaN values

# Print results
# Print results
print(f"NaN values found in {len(nan_positions)} locations.")
print(f"Columns with NaN values: {nan_columns.tolist()}")
print(f"First 10 NaN positions (row, col): {nan_positions[:10].tolist()}")  # Show first 10 for brevity

### Preprocessing using filtering

In [ ]:
from scipy.signal import butter, filtfilt, iirnotch

def bandpass_filter(data, lowcut, highcut, fs, order=7):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def notch_filter(data, notch_freq, fs, quality_factor=20):
    nyquist = 0.5 * fs
    notch = notch_freq / nyquist
    b, a = iirnotch(notch, quality_factor)
    y = filtfilt(b, a, data)
    return y

fs = 200  # sampling frequency in Hz
# Apply notch filter to suppress 50 Hz signal
filtered_RF = notch_filter(RF, 50, fs)
filtered_RH = notch_filter(RH, 50, fs)

# Apply bandpass filter between 0.5-30 Hz
filtered_signal_RF = bandpass_filter(RF, 0.5, 30, fs)
filtered_signal_RH = bandpass_filter(RH, 0.5, 30, fs)

In [ ]:
# Plot the raw and filtered signals
plt.figure(figsize=(12, 6))
plt.subplot(2, 2, 1)
plt.plot(RF[5000:6000,84])
plt.title('Raw EEG Signal')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

plt.subplot(2, 2, 2)
plt.plot(filtered_signal_RF[5000:6000,84])
plt.title('Filtered EEG Signal (Notch + Bandpass)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')


plt.subplot(2, 2, 3)
plt.plot(RH[5000:6000,84])
plt.title('Raw EEG Signal')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

plt.subplot(2, 2, 4)
plt.plot(filtered_signal_RH[5000:6000,84])
plt.title('Filtered EEG Signal (Notch + Bandpass)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

plt.tight_layout()
plt.show()


### Select any one channel for analysis and reshape the data into 10 second long epoch

#### Final obtained signal

i. 400 signals belongs to right hand class

ii. 500 signals belongs to right foot class

iii. Therefore, a total of 900 signals for right hand and right foot

In [ ]:
# Reshape the data and only select any one channel for simplicity
Right_hand = filtered_signal_RH[:,102] # Select any one channel but not the channel should be same for RH and RF
Right_foot = filtered_signal_RF[:,102] # Select any one channel but not the channel should be same for RH and RF

Right_hand = Right_hand.reshape(400, 1000) # Reshape the right hand data of second channel into 10 second (100 Hz *10 sec)
Right_foot = Right_foot.reshape(500, 1000) # Reshape the right foot data of second channel into 10 second (100 Hz *10 sec)

### Now create a binary labels indicating RH and RF

In [ ]:
Y = np.concatenate((np.zeros((400,1)), np.ones((500,1))), axis=0) 
X = np.concatenate((Right_hand,Right_foot)) # 0 indicate RH and 1 indicate RF

## This section indicate feature extraction 

#### Statistical Features

In [ ]:
import scipy.stats as stats

In [ ]:
# Statistical Features
def extract_statistical_features(signal):
    mean = np.mean(signal)
    median = np.median(signal)
    std_dev = np.std(signal)
    variance = np.var(signal)
    skewness = stats.skew(signal)
    kurtosis = stats.kurtosis(signal)
    range_val = np.ptp(signal)
    
    features = {
        'Mean': mean,
        'Median': median,
        'Standard Deviation': std_dev,
        'Variance': variance,
        'Skewness': skewness,
        'Kurtosis': kurtosis,
        'Range': range_val
    }
    
    return features

stat_features = [extract_statistical_features(signal) for signal in X]

#### Time Domain Features

In [ ]:
# Time Domain Features
def extract_time_domain_features(signal):
    rms = np.sqrt(np.mean(signal**2))
    zero_crossings = ((signal[:-1] * signal[1:]) < 0).sum()
    autocorrelation = np.correlate(signal, signal, mode='full')[len(signal)-1]
    mean_abs_dev = np.mean(np.abs(signal - np.mean(signal)))
    max_val = np.max(signal)
    min_val = np.min(signal)
    signal_energy = np.sum(signal**2)
    
    features = {
        'RMS': rms,
        'Zero Crossings': zero_crossings,
        'Autocorrelation': autocorrelation,
        'Mean Absolute Deviation': mean_abs_dev,
        'Max Value': max_val,
        'Min Value': min_val,
        'Signal Energy': signal_energy
    }
    
    return features

time_features = [extract_time_domain_features(signal) for signal in X]

#### Frequency Domain Features

In [ ]:
from scipy.signal import welch  # Import the welch function
# Frequency Domain Features
def extract_frequency_domain_features(signal):
    fs = 100
    freqs, psd = welch(signal, fs)
    dominant_freq = freqs[np.argmax(psd)]
    total_power = np.sum(psd)
    band_power = np.sum(psd[(freqs >= 0.5) & (freqs <= 40)])
    mean_freq = np.mean(freqs)
    median_freq = np.median(freqs)
    peak_freq = freqs[np.argmax(psd)]
    freq_variance = np.var(freqs)
    
    features = {
        'Dominant Frequency': dominant_freq,
        'Total Power': total_power,
        'Band Power (0.5-40 Hz)': band_power,
        'Mean Frequency': mean_freq,
        'Median Frequency': median_freq,
        'Peak Frequency': peak_freq,
        'Frequency Variance': freq_variance
    }
    
    return features

freq_features = [extract_frequency_domain_features(signal) for signal in X] # fs is a sampling rate

In [ ]:
import pywt
import antropy as ant

#### Entropy Features

In [ ]:
# Entropy Features
def extract_entropy_features(signal):
    fs = 100
    sample_entropy = ant.sample_entropy(signal)
    spectral_entropy = ant.spectral_entropy(signal, sf=fs, method='welch')
    perm_entropy = ant.perm_entropy(signal, normalize=True)
    svd_entropy = ant.svd_entropy(signal, order=3, delay=1)
    app_entropy = ant.app_entropy(signal)
    lziv_complexity = ant.lziv_complexity(signal)
    
    features = {
        'Sample Entropy': sample_entropy,
        'Spectral Entropy': spectral_entropy,
        'Permutation Entropy': perm_entropy,
        'SVD Entropy': svd_entropy,
        'Approximate Entropy': app_entropy,
        'LZiv Complexity': lziv_complexity
    }
    
    return features

entropy_features = [extract_entropy_features(signal) for signal in X] # fs is a sampling rate


### Feature concatenation 

In [ ]:
# Combine all features into a single DataFrame
def combine_features(stat_features, time_features, freq_features, entropy_features):
    combined_features_list = []
    for i in range(len(stat_features)):
        combined_features = {**stat_features[i], **time_features[i], **freq_features[i], **entropy_features[i], }
        combined_features_list.append(combined_features)
    return pd.DataFrame(combined_features_list)

combined_features_df = combine_features(stat_features, time_features, freq_features, entropy_features)
#print("Combined Features DataFrame:")
#print(combined_features_df)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create DataFrame
df = pd.DataFrame(combined_features_df)

# Initialize the StandardScaler
scaler = StandardScaler()

# Apply the scaler to normalize the data
normalized_data = scaler.fit_transform(df)

# Convert the result back into a DataFrame
normalized_df = pd.DataFrame(normalized_data, columns=combined_features_df.columns)
print(normalized_df)
print(normalized_df.columns)


### Feature selection

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

In [ ]:
def RF_fs(X,Y,n_features_to_select):
    # Initialize the model
    model = RandomForestClassifier()
    
    # Initialize RFE with the model and number of features to select
    rfe = RFE(model, n_features_to_select=n_features_to_select)
    
    # Fit RFE
    rfe = rfe.fit(X, Y)
    
    # Get the selected features
    selected_features = X.columns[rfe.support_]
    selected_features = selected_features.tolist()
    
    return selected_features

#selected_features = RF_fs(normalized_df,Y,2)
#print("Selected Features:", selected_features)

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

# === GA for Feature Selection ===
def genetic_algorithm(X, y, k=7, pop_size=10, generations=10, mutation_rate=0.2):
    n_features = X.shape[1]
    reg_par = 0.01  # Regularization parameter

    def initialize_population():
        return np.array([
            np.random.permutation([1]*k + [0]*(n_features - k)) for _ in range(pop_size)
        ])

    def fitness(ind):
        selected = np.where(ind == 1)[0]
        if len(selected) == 0:
            return 0
        scores = cross_val_score(SVC(kernel='rbf'), X.iloc[:, selected], y, cv=5)
        return scores.mean() - reg_par * (len(selected) / n_features)

    def crossover(p1, p2):
        merged = np.where((p1 + p2) > 0)[0]
        selected = np.random.choice(merged, k, replace=False)
        child = np.zeros(n_features, dtype=int)
        child[selected] = 1
        return child

    def mutate(ind):
        ones, zeros = np.where(ind == 1)[0], np.where(ind == 0)[0]
        if len(ones) > 0 and len(zeros) > 0:
            ind[np.random.choice(ones)] = 0
            ind[np.random.choice(zeros)] = 1
        return ind

    def select_tournament(pop, scores):
        participants = np.random.choice(len(pop), 3, replace=False)
        return pop[participants[np.argmax([scores[i] for i in participants])]]

    population = initialize_population()
    for _ in range(generations):
        scores = [fitness(ind) for ind in population]
        new_pop = []
        for _ in range(pop_size):
            p1 = select_tournament(population, scores)
            p2 = select_tournament(population, scores)
            child = crossover(p1, p2)
            if np.random.rand() < mutation_rate:
                child = mutate(child)
            new_pop.append(child)
        population = np.array(new_pop)

    final_scores = [fitness(ind) for ind in population]
    best = population[np.argmax(final_scores)]
    best = np.where(best == 1)[0]
    selected_features = X.columns[best]
    selected_features = selected_features.tolist()
    return selected_features

#selected_features = genetic_algorithm(normalized_df, Y, k=2, pop_size=10, generations=10, mutation_rate=0.2)

#print("Selected Features:", selected_features)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def cfs(X, y, max_features=7):
    y = LabelEncoder().fit_transform(y)  # Encode class labels numerically

    def cfs_merit(relevant_corr, redundant_corr, k):
        if k == 0:
            return 0
        numerator = k * relevant_corr
        denominator = np.sqrt(k + k * (k - 1) * redundant_corr)
        return numerator / denominator
    
    def compute_correlations(X, y):
        corr_with_target = []
        for col in X.columns:
            corr = np.abs(np.corrcoef(X[col], y)[0, 1])
            corr_with_target.append(corr)
        
        corr_with_target = np.array(corr_with_target)
        corr_matrix = np.abs(X.corr().values)  # Feature-feature correlations
        return corr_with_target, corr_matrix


    corr_target, corr_matrix = compute_correlations(X, y)
    
    selected = []
    remaining = list(range(X.shape[1]))
    best_merit = 0
    feature_names = X.columns.tolist()
    
    for _ in range(max_features):
        merits = []
        for i in remaining:
            test_subset = selected + [i]
            k = len(test_subset)
            rel = np.mean([corr_target[j] for j in test_subset])
            # Compute redundancy only if there are at least 2 features
            if len(test_subset) > 1:
                redundancy_vals = [corr_matrix[j][k] for j in test_subset for k in test_subset if j != k]
                red = np.nanmean(redundancy_vals) if redundancy_vals else 0
            else:
                red = 0  # No redundancy with only one feature
            merit = cfs_merit(rel, red, k)
            merits.append((i, merit))
        
        if merits:
            i_best, merit_best = max(merits, key=lambda x: x[1])
            selected.append(i_best)
            remaining.remove(i_best)
        else:
            break  # No further improvement
    
    return [feature_names[i] for i in selected]

#selected_features = cfs(normalized_df, Y,2)
#print("Selected features:", selected_features)

In [ ]:
from pyHSICLasso import HSICLasso

def HSICLasso_fs(X, Y, n_features_to_select=2):
    # Initialize HSIC Lasso
    hsic_lasso = HSICLasso()

    # Provide input data
    X_lasso = X.to_numpy()
    Y_lasso = Y.values.ravel() if isinstance(Y, pd.Series) else np.ravel(Y)

    # Provide input data
    hsic_lasso.input(X_lasso, Y_lasso, featname=X.columns.tolist())

    # Perform classification to select top n_features_to_select features
    hsic_lasso.classification(n_features_to_select)

    # Retrieve names of selected features (if feature names were provided)
    selected_features = hsic_lasso.get_features()
    
    return selected_features

#selected_features = HSICLasso_fs(normalized_df, Y, n_features_to_select=2)
#print("Selected feature names:", selected_features)


### Classification using SVM (linear and nonlinear)

In [ ]:
# Import SVM and train test split
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_matrix(cm, title='Confusion Matrix', folder_name=None):
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Right hand', 'Right foot'])
    disp.plot(cmap=plt.cm.Blues)
    plt.title(title)
    plt.savefig(folder_name + title)
    plt.show()

In [ ]:
def plot_decision_boundary(clf, X, y, name, feature_names=None, folder_name=None):
    # Convert X to NumPy array if it's a DataFrame
    if hasattr(X, "values"):
        X = X.values

    # Create a mesh grid
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500),
                         np.linspace(y_min, y_max, 500))

    # Predict on grid points
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Plot contours
    plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), Z.max(), 50), cmap=plt.cm.coolwarm, alpha=0.8)
    plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors='k')  # Decision boundary

    # Scatter plot of data points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm, edgecolors='k')
    plt.xlabel(feature_names[0] if feature_names else 'Feature 1')
    plt.ylabel(feature_names[1] if feature_names else 'Feature 2')
    plt.title(name)
    plt.savefig(folder_name + name)
    plt.show()

### ROC AUC Curve

In [ ]:
from sklearn.metrics import roc_curve, auc

def plot_roc_curve(y_test, y_pred, name='ROC Curve', folder_name=None):
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(10, 7))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(name)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(folder_name + name)
    plt.show()

### Performance assessment

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

def performance_assessment(filtered_signal_RH,filtered_signal_RF,fs_method, n_features=2, channels=[0,1], folder_path=None):

    def accuracy(confusion_matrix):
        TN = confusion_matrix[0, 0]
        TP = confusion_matrix[1, 1]
        FP = confusion_matrix[0, 1]
        FN = confusion_matrix[1, 0]
        return 100 * ((TP + TN) / (TP + TN + FP + FN)) if (TP + TN + FP + FN) != 0 else 0
    
    def precision(confusion_matrix):
        TP = confusion_matrix[1, 1]
        FP = confusion_matrix[0, 1]
        return 100 * (TP / (TP + FP)) if (TP + FP) != 0 else 0
    
    def specificiy(confusion_matrix):
        TN = confusion_matrix[0, 0]
        FP = confusion_matrix[0, 1]
        return 100 * (TN / (TN + FP)) if (TN + FP) != 0 else 0
    
    def sensitivity(confusion_matrix):
        TP = confusion_matrix[1, 1]
        FN = confusion_matrix[1, 0]
        return 100 * (TP / (TP + FN)) if (TP + FN) != 0 else 0
    
    def F1_score(precision, sensitivity):
        return (2 * (precision * sensitivity) / (precision + sensitivity)) if (precision + sensitivity) != 0 else 0
    
    # Define the grid of hyperparameters to search
    param_grid = {
        'C': [2, 10, 100, 1000],  # Regularization parameter
        'gamma': [1, 0.5, 0.1],  # Kernel coefficient
        'kernel': ['rbf']  # Kernel type
    }
    
    for channel in range(channels[0]-1, channels[1]):

        # Reshape the data and only select any one channel for simplicity
        Right_hand = filtered_signal_RH[:,channel] # Select any one channel but not the channel should be same for RH and RF
        Right_foot = filtered_signal_RF[:,channel] # Select any one channel but not the channel should be same for RH and RF

        Right_hand = Right_hand.reshape(400, 1000) # Reshape the right hand data of second channel into 10 second (100 Hz *10 sec)
        Right_foot = Right_foot.reshape(500, 1000) # Reshape the right foot data of second channel into 10 second (100 Hz *10 sec)

        Y = np.concatenate((np.zeros((400,1)), np.ones((500,1))), axis=0) 
        X = np.concatenate((Right_hand,Right_foot)) # 0 indicate RH and 1 indicate RF

        # Feature extraction
        stat_features = [extract_statistical_features(signal) for signal in X]
        time_features = [extract_time_domain_features(signal) for signal in X]
        freq_features = [extract_frequency_domain_features(signal) for signal in X] # fs is a sampling rate
        entropy_features = [extract_entropy_features(signal) for signal in X] # fs is a sampling rate

        # Combine all features into a single DataFrame
        combined_features_df = combine_features(stat_features, time_features, freq_features, entropy_features)
        
        # Create DataFrame and normalize the data
        df = pd.DataFrame(combined_features_df)
        scaler = StandardScaler()
        normalized_data = scaler.fit_transform(df)
        normalized_df = pd.DataFrame(normalized_data, columns=combined_features_df.columns)
        
        # Define 5-fold cross-validation
        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        # Initialize a matrix to store the combined confusion matrix
        combined_confusion_matrix = np.zeros((len(np.unique(Y)), len(np.unique(Y))), dtype=int)

        # Initialize lists to store all true labels and predicted probabilities
        y_true_combined = []
        y_pred_combined = []

        # Perform cross-validation manually to access each fold
        for fold, (train_index, test_index) in enumerate(kf.split(normalized_df), start=1):
            X_train, X_test = normalized_df.iloc[train_index], normalized_df.iloc[test_index]
            y_train, y_test = Y[train_index], Y[test_index]

            selected_features = fs_method(X_train, y_train, n_features)
            X_selected_train = X_train[selected_features]

            # Define the SVM model
            svm = SVC(probability=True)

            # Perform grid search with cross-validation
            grid_search = GridSearchCV(svm, param_grid, cv=10, scoring='balanced_accuracy')
            grid_search.fit(X_selected_train, y_train)
            best_svm = grid_search.best_estimator_
            
            X_selected_test = X_test[selected_features]
            # Get probability predictions for ROC curve
            y_pred = best_svm.predict(X_selected_test)
            
            # Append true labels and predicted probabilities to combined lists
            y_true_combined.extend(y_test)
            y_pred_combined.extend(y_pred)
            
            # Calculate confusion matrix for this fold
            cm = confusion_matrix(y_test, y_pred)
            
            # Combine confusion matrix (sum them)
            combined_confusion_matrix += cm

            # Plot decision boundary
            plot_decision_boundary(best_svm, X_selected_train, y_train, f'SVM Decision Boundary - Channel {channel + 1} - Fold {fold}', feature_names=selected_features, folder_name=folder_path)

        # Plot confusion matrix for the combined results
        plot_confusion_matrix(combined_confusion_matrix, title=f'Confusion Matrix - Channel {channel + 1}', folder_name=folder_path)
        # Plot ROC curve
        plot_roc_curve(y_true_combined, y_pred_combined, name=f'ROC Curve - Channel {channel + 1}', folder_name=folder_path)

        # Calculate performance metrics
        accuracy_score = accuracy(combined_confusion_matrix)
        precision_score = precision(combined_confusion_matrix)
        specificity_score = specificiy(combined_confusion_matrix)
        sensitivity_score = sensitivity(combined_confusion_matrix)
        f1_score = F1_score(precision_score, sensitivity_score)
        print(f"Performance Metrics for Channel {channel + 1}:")
        print(f"Accuracy: {accuracy_score:.4f}")
        print(f"Precision: {precision_score:.4f}")
        print(f"Specificity: {specificity_score:.4f}")
        print(f"Sensitivity: {sensitivity_score:.4f}")
        print(f"F1 Score: {f1_score:.4f}")
        print("-" * 50)

        # Save data in file
        with open(folder_path + f'performance_metrics_channel_{channel + 1}.txt', 'w') as f:
            f.write(f"Performance Metrics for Channel {channel + 1}:\n")
            f.write(f"Accuracy: {accuracy_score:.4f}\n")
            f.write(f"Precision: {precision_score:.4f}\n")
            f.write(f"Specificity: {specificity_score:.4f}\n")
            f.write(f"Sensitivity: {sensitivity_score:.4f}\n")
            f.write(f"F1 Score: {f1_score:.4f}\n")
            f.write("-" * 50 + "\n")

    
    return 

In [ ]:

performance_assessment(filtered_signal_RH,filtered_signal_RF,cfs, n_features=2, channels=[1,20], folder_path='cfs_results/')

In [ ]:
performance_assessment(filtered_signal_RH,filtered_signal_RF,RF_fs, n_features=2, channels=[1,20], folder_path='rf_results/')

In [ ]:
performance_assessment(filtered_signal_RH,filtered_signal_RF,HSICLasso_fs, n_features=2, channels=[1,20], folder_path='HSIC_Lasso_results/')

In [ ]:
performance_assessment(filtered_signal_RH,filtered_signal_RF,genetic_algorithm, n_features=2, channels=[1,20], folder_path='GA_results/')